In [2]:
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# Polynomial GP

In [19]:
def build_covariance(x, xp, kern):
    """Build a covariance matrix
    Inputs
    -------
    x: (N, 2) array of inputs
    xp: (M, 2) array of inptus
    kern: a function mapping inputs to covariance
    Outputs
    -------
    cov: (N, M) covariance matrix
    """
    out = np.zeros((x.shape[0], xp.shape[0]))
    for jj in range(xp.shape[0]):
        out[:, jj] = kern(x, xp[jj])
    return out


In [20]:
def gpr(xtrain, ytrain, xpred, noise_var, mean_func, kernel):
    """Gaussian process regression Algorithm

    Inputs
    -------
    xtrain: (N, ) training inputs
    ytrain: (N, ) training outputs
    xpred:  (M, ) locations at which to make predictions
    noise_var: (N, ) noise at every training output
    mean_func: function to compute the prior mean
    kernel: covariance kernel
    
    Returns
    -------
    pred_mean : (M, ) predicted mean at prediction points
    pred_cov : (M, M) predicted covariance at the prediction points
    --
    """
    cov = build_covariance(xtrain, xtrain, kernel)
    u, s, v = np.linalg.svd(cov)
    sqrtcov = np.dot(u, np.sqrt(np.diag(s)))

    # pseudoinverse is better conditioned
    invcov = np.linalg.pinv(cov + np.diag(noise_var))

    vec_pred = build_covariance(xpred, xtrain, kernel)
    pred_mean = mean_func(xpred) + np.dot(vec_pred, np.dot(invcov, ytrain - mean_func(xtrain)))
    
    cov_predict_pre = build_covariance(xpred, xpred, kernel)
    cov_predict_up = np.dot(vec_pred, np.dot(invcov, vec_pred.T))
    pred_cov = cov_predict_pre - cov_predict_up
    
    return pred_mean, pred_cov

In [21]:
def sqexp(x, xp, tau=1, l=0.5):
    """Squared exponential kernel (1 dimensional)
    
    Inputs
    ------
    x : (N), array of multiple inputs
    xp: float
    
    Returns
    -------
    cov (N,) -- Covariance between each input at *x* and the function values at *x*
    """
    cov = tau**2 * np.exp(-1/2 * (x - xp)**2 / l**2)
    return cov

def periodic(x, xp, tau=1, l=1.0, p=0.4):
    """Periodic kernel"""
    return tau**2 * np.exp ( -2 * np.sin(np.pi * np.abs(x - xp) / p )**2 / l**2)

def poly(x, xp, c=1, d=3):
    """Polynomial kernel"""
    return (x * xp + c)**d
    
KERNELS = {"Sq.Exp":sqexp, "Periodic":periodic, "Poly":poly}

In [22]:
def plot_function_gp(axis, pred_mean, predict_cov, xdata, ydata, xpred, ytrue, title, label):
    axis.plot(xdata, ydata, 'o', ms=10, label='data')
    axis.plot(xpred, ytrue, '-', lw=3, color='black', label='true function')
    axis.plot(xpred, pred_mean, 'r--', lw=3, label=label)
    pred_fstd = np.sqrt(np.diag(predict_cov))
    axis.fill_between(xpred, pred_mean - 2*pred_fstd, pred_mean+2*pred_fstd, lw=3, label=r'2$\sigma$', color='red', alpha=0.2)
    axis.legend()
    axis.set_title(title, fontsize=14)

In [28]:
def true_function(x):
    #return np.exp(-x**2 / 0.1)*np.sin(10*x) #- x**3
    return 5*x**5*np.sin(20*x) + 0.2*x**6 - x - 0.5
    # return np.sin(9.5 * np.pi * x) + 10.0

XSPACE = np.linspace(-1,1,300)
NUM_DATA = 20
NOISE_COV = 1e-3
#NOISE_COV = 1e-2

#XDATA = np.random.rand(NUM_DATA)*2.0 - 1.0 # randomly spaced data
XDATA = np.linspace(-1, 1, NUM_DATA) # linearly space data
YDATA = true_function(XDATA) + np.random.randn(NUM_DATA)*np.sqrt(NOISE_COV)
YTRUE = true_function(XSPACE)   

In [36]:
def gp_regression_demo(mean_func, kernel, xdata, ydata, xtrue, ytrue, noise_cov=1e-1):
    
    prior_mean = mean_func(xtrue)
    print(xtrue.shape)
    prior_cov = build_covariance(xtrue, xtrue, kernel)
    fig, axis = plt.subplots(1, 2, figsize=(15, 5))
    plot_function_gp(axis[0], prior_mean, prior_cov, xdata, ydata, xtrue, ytrue, "Prior", "Prior mean")
    
    # perform inference
    mean_predict, cov_predict = gpr(xdata, ydata, xtrue, noise_cov * np.ones((xdata.shape[0])), mean_func, kernel)
    print(mean_predict.shape)
    plot_function_gp(axis[1], mean_predict, cov_predict, xdata, ydata, xtrue, ytrue, "Posterior", "Posterior mean")
    plt.show()

# always use zero mean
MEANF = lambda x: np.zeros((x.shape[0]))
#MEANF = lambda x: 10*x**2
#MEANF = lambda x: true_function(x)

In [33]:
# Squared Exponential Kernel
@interact_manual(tau=(0.5,2,0.2), log_width=(-4, 1, 0.1), log_noise_learn=(-10,0,1))
def sqexp_infer(tau=1, log_width=-1, log_noise_learn=np.log10(NOISE_COV)):
    kernel_tot = KERNELS['Sq.Exp']
    def kernel(x, xp):
        return kernel_tot(x, xp, tau=tau, l=10**log_width)
    gp_regression_demo(MEANF, kernel, XDATA, YDATA, XSPACE, YTRUE, noise_cov=10**log_noise_learn)

interactive(children=(FloatSlider(value=1.0, description='tau', max=2.0, min=0.5, step=0.2), FloatSlider(value…

In [37]:
# Polynomial Kernel
@interact_manual(c=(0,1,0.1), d=(2, 16, 1), log_noise_learn=(-10,0,1))
def poly_infer(c=0, d=-1, log_noise_learn=np.log10(NOISE_COV)):
    kernel_tot = KERNELS['Poly']
    def kernel(x, xp):
        return kernel_tot(x, xp, c=c, d=d)
    gp_regression_demo(MEANF, kernel, XDATA, YDATA, XSPACE, YTRUE, noise_cov=10**log_noise_learn)

interactive(children=(FloatSlider(value=0.0, description='c', max=1.0), IntSlider(value=2, description='d', ma…

# Generate Pendulum

In [ ]:
def true_function(x1,x2,u):
    #return np.exp(-x**2 / 0.1)*np.sin(10*x) #- x**3
    # return 5*x**5*np.sin(20*x) + 0.2*x**6 - x - 0.5
    #return np.sin(9.5 * np.pi * x) + 10.0
    return 10/10*x1-1/(1*1.4**2)*x2+1/(1*1.4**2)*u